# Defining a polypharmacy variable in MIMIC-III

Polypharmacy refers to the concurrent use of multiple medications. This notebook aims to define a binary polypharmacy variable for hospital stays using the prescriptions table. Here, polypharmacy is defined for a hospital stay ('hadm_id') as the concurrent prescription of 5 or more drugs for the same day.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
from IPython.display import display, HTML # used to print out pretty pandas dataframes
import matplotlib.dates as dates
import matplotlib.lines as mlines
from itertools import combinations
import datetime
import re
import drugstandards as drugs

%matplotlib inline
plt.style.use('ggplot') 

# specify user/password/where the database is
sqluser = 'postgres'
sqlpass = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'
host = 'localhost'

query_schema = 'SET search_path to ' + schema_name + ';'

# connect to the database
con = psycopg2.connect(dbname=dbname, user=sqluser, password=sqlpass, host=host)

In [2]:
query = query_schema + """
SELECT subject_id, hadm_id, icustay_id, drug_type, drug, formulary_drug_cd, route, startdate, enddate
FROM prescriptions
"""
prescriptions = pd.read_sql_query(query, con)

C:\Users\manue\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


The number of unique drugs in the prescriptions table:

In [3]:
prescriptions['drug'].nunique()

4525

### Drug prescriptions of type 'BASE' (shown below) are excluded from the calculation of polypharmacy based on expert advice. These are mostly items prescribed to make up for deficiencies in a hospital setting.</font>

In [4]:
prescriptions[prescriptions['drug_type']=='BASE']['drug'].unique()

array(['D5W', 'Iso-Osmotic Dextrose', 'SW', 'Send 500mg Vial',
       'Syringe (Neonatal) *D5W*', 'NS', 'Soln.', 'NS (Glass Bottle)',
       '1/2 NS', 'D5 1/2NS', 'Syringe (IV Room)', 'Vial', 'Dextrose 5%',
       'Sterile Water', 'D5W (EXCEL BAG)', 'NS (Mini Bag Plus)', 'Soln',
       'LR', 'Syringe', 'Syringe (Neonatal)', 'D10W',
       'Potassium Chl 20 mEq / 1000 mL D5 1/2 NS',
       'Potassium Chl 20 mEq / 1000 mL NS', 'Isotonic Sodium Chloride',
       'Iso-Osmotic Sodium Chloride', '0.9% Sodium Chloride',
       '5% Dextrose', 'Amino Acids 4.25% W/ Dextrose 5%',
       'Syringe (Neonatal) *SW*', '0.45% Sodium Chloride',
       'Potassium Chl 20 mEq / 1000 mL D5NS', 'Solution', 'D5NS',
       'Sodium Chloride 3% (Hypertonic)', 'D5W (Excel)',
       'NS Epidural Bag (0.9% NaCl)', 'Bag', 'AMP', 'Syringe (PCA)',
       'D5W (GLASS BOTTLE)', 'Starter PN D10', 'Prismasate (B32 K2)',
       'D12.5W', 'Potassium Chl 40 mEq / 1000 mL D5 1/2 NS',
       '0.9% Sodium Chloride (Mini Bag Pl

In [5]:
prescriptions_to_include = prescriptions[prescriptions['drug_type']!='BASE']

### Some drugs of drug_type 'BASE' also fall under drug_type 'MAIN'. These drugs are also exluded when creating the polypharmacy variable.

In [6]:
bases = prescriptions[prescriptions['drug_type']=='BASE']['drug'].unique().tolist()
mains = prescriptions[prescriptions['drug_type']=='MAIN']['drug'].unique().tolist()
bases_in_mains = list(set(mains) & set(bases))
bases_in_mains

['Prismasate (B32 K2)',
 'Sodium CITRATE 4%',
 'Citrate Dextrose 3% (ACD-A) CRRT',
 'Intralipid 20%',
 'Prismasate (B22 K4)',
 'Syringe (Neonatal)',
 'PrismaSol  (B32 K0)']

In [7]:
prescriptions_to_include = prescriptions_to_include[~prescriptions_to_include['drug'].isin(bases_in_mains)]

### From the drug prescriptions of type 'ADDITIVE', a set of drugs were excluded in the calculation of polypharmacy based on expert advice.

In [8]:
additives = prescriptions[prescriptions['drug_type']=='ADDITIVE']['drug'].tolist()
prescriptions[prescriptions['drug_type']=='ADDITIVE']['drug'].unique()

array(['Calcium Gluconate', 'Potassium Chloride', 'Sodium Chloride ',
       'Sodium Acetate', 'Heparin (Preservative Free)',
       'Potassium Acetate', 'Multivitamin 12', 'Folic Acid',
       'Multivitamin-12', 'Thiamine HCl', 'Sodium Chloride',
       'Hydrocortisone Na Succ.', 'Heparin Sodium', 'Bupivacaine 0.5%',
       'Epoetin Alfa', 'Sodium Chloride 23.4%', 'Sodium Bicarbonate',
       'Multivitamin IV', 'Sodium Chloride 0.9%',
       'Hydrocortisone Na Succinate', 'Heparin (Preserv. Free)',
       'Bupivacaine 0.50%', 'Lidocaine 1% PF', 'Potassium Phosphate',
       'Octreotide Acetate', 'Heparin', 'Mannitol', 'Lidocaine 1% P.F.',
       'Magnesium Sulfate', 'Thiamine', 'Mesna', 'Fentanyl Citrate',
       'Dextrose 50%', 'Albumin 25% (12.5gm)', 'Calcium Chloride',
       'Cytarabine', 'VinCRIStine', 'Humulin-R Insulin', 'Mannitol 20%',
       'Multivitamin IV (Without Vit. K)', 'Bupivacaine 0.05%',
       'Bupivacaine 0.1%', 'Insulin Human Regular', 'Phytonadione',
       'Fam

In [9]:
additives_to_include = ['Calcium Gluconate', 'Potassium Chloride', 'Heparin (Preservative Free)',
       'Multivitamin 12', 'Folic Acid',
       'Multivitamin-12', 'Thiamine HCl',
       'Hydrocortisone Na Succ.', 'Heparin Sodium', 'Bupivacaine 0.5%',
       'Epoetin Alfa', 'Sodium Bicarbonate',
       'Multivitamin IV',
       'Hydrocortisone Na Succinate', 'Heparin (Preserv. Free)',
       'Bupivacaine 0.50%', 'Lidocaine 1% PF', 'Potassium Phosphate',
       'Octreotide Acetate', 'Heparin', 'Mannitol', 'Lidocaine 1% P.F.',
       'Magnesium Sulfate', 'Thiamine', 'Mesna', 'Fentanyl Citrate', 'Calcium Chloride',
       'Cytarabine', 'VinCRIStine', 'Humulin-R Insulin',
       'Multivitamin IV (Without Vit. K)', 'Bupivacaine 0.05%',
       'Bupivacaine 0.1%', 'Insulin Human Regular', 'Phytonadione',
       'Famotidine', 'Vincristine Sulfate', 'Lidocaine 1%', 'CloniDINE', 'Sodium Phosphate',
       'Clonidine HCl', 'Hydromorphone P.F.', 'KCl',
       'Hydrocortisone Sod Succinate', 'FoLIC Acid',
       'VinCRIStine (Oncovin)', 'Hydrocortisone Na Succ', 'Ondansetron',
       'Fosaprepitant', 'Hydrocortisone', 'Bupivacaine 0.2%',
       'HYDROmorphone P.F.', 'Methotrexate Sodium P.F.', 'Lidocaine 2%',
       'Hepatitis B Immu Glob (HepaGam B)', 'DOXOrubicin']

In [10]:
additives_to_exclude = list(set(additives)-set(additives_to_include))
additives_to_exclude

['Sodium Chloride',
 'Sodium Acetate',
 'Normocarb Dialysate',
 'Mannitol 20%',
 'Albumin',
 'Sodium Chloride 23.4%',
 'Dextrose 50%',
 'Sodium Chloride 0.9%',
 'Sodium Chloride ',
 'Albumin 25% (12.5gm)',
 'Potassium Acetate',
 'Albumin 25% (12.5g / 50mL)']

In [11]:
prescriptions_to_include = prescriptions_to_include[~prescriptions_to_include['drug'].isin(additives_to_exclude)]

### Drugs taken through certain routes were excluded based on expert advice.

In [12]:
routes_to_include = ['PO', 'IV', 'IV DRIP', 'SC', 'SL', 'IM', 'NG', 'PR', 'OU', 'IVPCA',
       'IH', 'TP', 'DIALYS', 'NU', 'ORAL', 'IV BOLUS', 'PO/NG', 'TD',
       'BOTH EYES', 'PB', 'ED', 'REPLACE', 'G TUBE', 'ET', 'DWELL', 'IR',
       'VG', 'LEFT EYE', 'NEB', 'IN', 'ID', 'AU', 'OS', 'IRR', 'OD',
       'INHALATION', 'IA', 'AS', 'AD', 'J TUBE', 'LOCK', 'IT', 'IJ',
       'IP', 'PL', 'LEFT EAR', 'NAS', 'TT', 'RIGHT EYE', 'EX-VIVO', 'NS',
       'BU', 'BOTH EARS', 'BUCCAL', 'SUBCUT', 'SCPUMP', 'ND',
       'PERIPHNERVE', 'AERO', 'IO', 'VT', 'RIGHT EAR', 'PO/IV', 'OG',
       'PO/PR', 'RECTAL', 'SCPCA', 'IVT', 'IC', 'IVS', 'NG/OG', 
       'ENTERAL TUBE ONLY ? NOT ORAL', 'PO OR ENTERAL TUBE', 'PO/OG']
few_routes_only = ['PO/IV', 'PO', 'IV', 'SC', 'SUBCUT', 'ORAL', 'IV DRIP', 'IM', 'NG', 'PO/NG', 'IVPCA', 
                   'IV BOLUS', 'IVT', 'IVS']
# Use few_routes_only if analysis becomes too cumbersome if all routes in routes_to_include are used

In [13]:
routes = prescriptions['route'].unique().tolist()
prescriptions['route'].unique()

array(['PO', 'IV', 'IV DRIP', 'SC', 'SL', 'IM', 'NG', 'PR', 'OU', 'IVPCA',
       'IH', 'TP', 'DIALYS', 'NU', 'ORAL', 'IV BOLUS', 'PO/NG', 'TD',
       'BOTH EYES', 'PB', 'ED', 'REPLACE', 'G TUBE', 'ET', 'DWELL', 'IR',
       'VG', 'LEFT EYE', 'NEB', 'IN', 'ID', 'AU', 'OS', 'IRR', 'OD',
       'INHALATION', 'IA', 'AS', 'AD', 'J TUBE', 'LOCK', 'IT', None, 'IJ',
       'IP', 'PL', 'LEFT EAR', 'NAS', 'TT', 'RIGHT EYE', 'EX-VIVO', 'NS',
       'BU', 'BOTH EARS', 'BUCCAL', 'SUBCUT', 'SCPUMP', 'ND',
       'PERIPHNERVE', 'AERO', 'IO', 'VT', 'RIGHT EAR', 'PO/IV', 'OG',
       'PO/PR', 'RECTAL', 'SCPCA', 'IVT', 'IC', 'IVS', 'NG/OG',
       'ENTERAL TUBE ONLY ? NOT ORAL', 'PO OR ENTERAL TUBE', 'PO/OG',
       'INTERSPACE', 'INTRAPERICARDIAL', 'LUMBAR PLEXUS', 'AXILLARY'],
      dtype=object)

In [14]:
#routes_to_exclude = list(set(routes)-set(routes_to_include))
routes_to_exclude = list(set(routes)-set(few_routes_only))
print(routes_to_exclude)
if None in routes_to_exclude: 
    routes_to_exclude.remove(None)
if np.nan in routes_to_exclude: 
    routes_to_exclude.remove(np.nan)
prescriptions_to_include = prescriptions_to_include[~prescriptions_to_include['route'].isin(routes_to_exclude)]

['PO OR ENTERAL TUBE', 'PB', 'RECTAL', 'ET', 'PL', 'ND', 'LEFT EAR', 'BOTH EYES', 'NU', 'OD', 'TP', 'NAS', 'OU', 'NEB', 'EX-VIVO', 'LEFT EYE', 'NG/OG', 'AU', 'IRR', 'RIGHT EAR', 'OG', 'RIGHT EYE', 'BOTH EARS', 'TT', 'BU', 'LOCK', 'PR', None, 'NS', 'G TUBE', 'ED', 'IT', 'IR', 'AERO', 'ID', 'SCPCA', 'IJ', 'INHALATION', 'BUCCAL', 'SCPUMP', 'DIALYS', 'OS', 'PO/PR', 'INTRAPERICARDIAL', 'PO/OG', 'IP', 'PERIPHNERVE', 'IA', 'VT', 'VG', 'AXILLARY', 'INTERSPACE', 'IC', 'DWELL', 'IH', 'TD', 'J TUBE', 'ENTERAL TUBE ONLY ? NOT ORAL', 'SL', 'AS', 'IO', 'AD', 'LUMBAR PLEXUS', 'REPLACE', 'IN']


In [15]:
prescriptions_to_include['drug'].nunique()

3497

The above steps have reduced the number of different drugs from 4525 to 3497.

<font size="4">The entries in the 'drug' column have many values that are duplicates due to either spelling differences, different dosages, different cases, different brand names etc. In calculating polypharmacy, we are only concerned about the drug name, we do not consider different doses or brand names as different drugs. Due to this, we try to reduce the duplicate entries as much as possible before calculating polypharmacy. The first step is to make the case uniform and remove numbers and special characters from the entries in the 'drug' column. In the next step, we use the 'formulary_drug_cd' column and map the entries to the entries in the 'drug' column. This helps in reducing the 'drug' column as there are fewer unique entries in the 'formulary_drug_cd' column than the 'drug' column. We have ignored null values present in the 'formulary_drug_cd' column. Finally, we standardise the entries in the 'drug' column using a package called drugstandards https://doi.org/10.5281/zenodo.571248. Some entries in the 'drug' column may be standardised to null values if no suitable match is found. We ignore these emtries in the calculation of polypharmacy.</font>

### Simplifying the 'drug' column by making drug names to lower case, keeping only alphabet characters in the drug names and removing any words of length less than 3.

In [16]:
prescriptions_to_include['drug'] = prescriptions_to_include['drug'].str.lower()
prescriptions_to_include['drug'] = prescriptions_to_include['drug'].str.replace(r'[^a-zA-Z]+', r' ')
prescriptions_to_include['drug'] = prescriptions_to_include['drug'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
prescriptions_to_include['drug'] = prescriptions_to_include['drug'].str.replace(r'\b\w\w\b', '').str.replace(r'\s+', ' ')
#removing multiple white spaces
prescriptions_to_include['drug'] = prescriptions_to_include['drug'].str.replace(r' +', r' ')
#removing white spaces at the ends
prescriptions_to_include['drug'] = prescriptions_to_include['drug'].str.strip()
prescriptions_to_include = prescriptions_to_include[prescriptions_to_include['drug']!='']

C:\Users\manue\AppData\Local\Temp/ipykernel_15608/259907385.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  prescriptions_to_include['drug'] = prescriptions_to_include['drug'].str.replace(r'[^a-zA-Z]+', r' ')
C:\Users\manue\AppData\Local\Temp/ipykernel_15608/259907385.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  prescriptions_to_include['drug'] = prescriptions_to_include['drug'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
C:\Users\manue\AppData\Local\Temp/ipykernel_15608/259907385.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  prescriptions_to_include['drug'] = prescriptions_to_include['drug'].str.replace(r'\b\w\w\b', '').str.replace(r'\s+', ' ')
C:\Users\manue\AppData\Local\Temp/ipykernel_15608/259907385.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  prescript

In [17]:
prescriptions_to_include['drug'].nunique()

2603

This has reduced the number of different drugs from 3497 to 2603.

### Reducing the 'formulary_drug_cd' column by making the case uniform, keeping only alphabet characters and removing single character words. This is done so that a mapping from 'formulary_drug_cd' to 'drug' can be done in order to further simplify the 'drug' column.

In [18]:
prescriptions_to_include['formulary_drug_cd'].nunique()

2595

In [19]:
prescriptions_to_include['formulary_drug_cd'] = prescriptions_to_include['formulary_drug_cd'].str.replace(r'[^a-zA-Z]+', r' ')
prescriptions_to_include['formulary_drug_cd'] = prescriptions_to_include['formulary_drug_cd'].str.upper()
prescriptions_to_include['formulary_drug_cd'] = prescriptions_to_include['formulary_drug_cd'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
#removing multiple white spaces
prescriptions_to_include['formulary_drug_cd'] = prescriptions_to_include['formulary_drug_cd'].str.replace(r' +', r' ')
#removing white spaces at the ends
prescriptions_to_include['formulary_drug_cd'] = prescriptions_to_include['formulary_drug_cd'].str.strip()

C:\Users\manue\AppData\Local\Temp/ipykernel_15608/196274087.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  prescriptions_to_include['formulary_drug_cd'] = prescriptions_to_include['formulary_drug_cd'].str.replace(r'[^a-zA-Z]+', r' ')
C:\Users\manue\AppData\Local\Temp/ipykernel_15608/196274087.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  prescriptions_to_include['formulary_drug_cd'] = prescriptions_to_include['formulary_drug_cd'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
C:\Users\manue\AppData\Local\Temp/ipykernel_15608/196274087.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  prescriptions_to_include['formulary_drug_cd'] = prescriptions_to_include['formulary_drug_cd'].str.replace(r' +', r' ')


In [20]:
prescriptions_to_include['formulary_drug_cd'].nunique()

1358

This has reduced the number of unique entries in the 'formulary_drug_cd' column from 2595 to 1358.

###  The number of unique values in the 'formulary_drug_cd' column is lower than the number in the 'drug' column. Thus, mapping from 'formulary_drug_cd' to 'drug' as shown below helps in futrther reducing the 'drug' column.

In [21]:
prescriptions_to_include['drug'].nunique()

2603

In [22]:
form_drug_codes = prescriptions_to_include['formulary_drug_cd'].unique().tolist()
form_drug_codes = [x for x in form_drug_codes if x != None and x!='' and x!=np.nan]
for drug_code in form_drug_codes:
    if len(prescriptions_to_include[prescriptions_to_include['formulary_drug_cd']==drug_code]['drug'].value_counts().tolist()) > 0:
        drug_name = prescriptions_to_include[prescriptions_to_include['formulary_drug_cd']==drug_code]['drug'].value_counts().index[0]
        prescriptions_to_include.loc[prescriptions_to_include['formulary_drug_cd']==drug_code, 'drug'] = drug_name

In [23]:
prescriptions_to_include['drug'].nunique()

1551

The mapping has reduced number of different drugs from 2603 to 1551.

### The drugstandards package is used to standardise the drug names. The package considers synonyms, brand names and also matches mispelled entries based on the Jaro-Winkler similarity. Using this package, a dictionary is created that maps unique entries in the 'drug' column to standardised drug names. This dictionary is then used on the 'drug' column.

In [24]:
drugs_list = prescriptions_to_include['drug'].unique().tolist()
std_drugs_list = drugs.standardize(drugs_list)
drugs_dict = dict(zip(drugs_list, std_drugs_list))
prescriptions_to_include.rename(columns={"drug": "std_drug"}, inplace = True)
prescriptions_to_include['drug'] = prescriptions_to_include['std_drug']
prescriptions_to_include.replace({'std_drug': drugs_dict}, inplace = True)

In [34]:
prescriptions_to_include.head()

,subject_id,hadm_id,icustay_id,drug_type,std_drug,formulary_drug_cd,route,startdate,enddate,drug
0,6,107064,NaN,MAIN,TACROLIMUS,TACR,PO,2175-06-11,2175-06-12,tacrolimus
1,6,107064,NaN,MAIN,WARFARIN,WARF,PO,2175-06-11,2175-06-12,warfarin
2,6,107064,NaN,MAIN,HEPARIN,HEPAPREMIX,IV,2175-06-11,2175-06-12,heparin sodium
4,6,107064,NaN,MAIN,FUROSEMIDE,FURO,PO,2175-06-11,2175-06-12,furosemide
5,6,107064,NaN,MAIN,WARFARIN,WARF,PO,2175-06-11,2175-06-15,warfarin


<font size="4">Polypharmacy is calculated for each 'hadm_id' by checking whether at least five different drugs are prescribed for the same day. To achieve this, firstly a list of dates is generated for each drug in the prescriptions table using the 'startdate' and 'enddate'. Following this, entries with null values are removed. Finally, for a given 'hadm_id' we check if there are at least five overlapping days among the lists of dates for different drugs that are prescribed. This would indicate if there are at least five different drugs prescribed for the same day.</font>

### Adding the list of dates from startdate to enddate for each prescription and removing prescriptions with null values.

In [35]:
dates = []
for i, row in prescriptions_to_include.iterrows():
    dates_i = []
    if row['startdate']:
        start = pd.to_datetime(row['startdate'])
        dates_i.append(start)
        if row['enddate']:
            end = pd.to_datetime(row['enddate'])
            if end-start>datetime.timedelta(days=0) and end-start<datetime.timedelta(days=200):
                while start!=end:
                    start = start + datetime.timedelta(days=1)
                    dates_i.append(start)
        dates.append(dates_i)
    else:
        dates.append(0)

In [36]:
prescriptions_to_include['dates'] = dates
prescriptions_to_include = prescriptions_to_include[prescriptions_to_include['std_drug'].notna()]
prescriptions_to_include = prescriptions_to_include[prescriptions_to_include['startdate'].notna()]
prescriptions_to_include = prescriptions_to_include[prescriptions_to_include['enddate'].notna()]

In [37]:
prescriptions_to_include.head()

,subject_id,hadm_id,icustay_id,drug_type,std_drug,formulary_drug_cd,route,startdate,enddate,drug,dates
0,6,107064,NaN,MAIN,TACROLIMUS,TACR,PO,2175-06-11,2175-06-12,tacrolimus,"[2175-06-11 00:00:00, 2175-06-12 00:00:00]"
1,6,107064,NaN,MAIN,WARFARIN,WARF,PO,2175-06-11,2175-06-12,warfarin,"[2175-06-11 00:00:00, 2175-06-12 00:00:00]"
2,6,107064,NaN,MAIN,HEPARIN,HEPAPREMIX,IV,2175-06-11,2175-06-12,heparin sodium,"[2175-06-11 00:00:00, 2175-06-12 00:00:00]"
4,6,107064,NaN,MAIN,FUROSEMIDE,FURO,PO,2175-06-11,2175-06-12,furosemide,"[2175-06-11 00:00:00, 2175-06-12 00:00:00]"
5,6,107064,NaN,MAIN,WARFARIN,WARF,PO,2175-06-11,2175-06-15,warfarin,"[2175-06-11 00:00:00, 2175-06-12 00:00:00, 217..."


### Checking for overlapping dates to create the polypharmacy variable.

In [ ]:
limit = 5
polypharm = []
for case_id in prescriptions_to_include['hadm_id'].unique():
    poly = [case_id]
    case = prescriptions_to_include[prescriptions_to_include['hadm_id'] == case_id][['hadm_id', 'std_drug', 'dates']].reset_index(drop=True)
    
    # combine all the same drugs into one entry simply including all the dates, then use combinations to check polypharmacy
    drugs = [index for index, value in case['std_drug'].value_counts().items() if value>1]
    if len(drugs)>0:
        for drug in drugs:
            drug_dates = [value for index, value in case[case['std_drug']==drug]['dates'].items()]
            flat_dates = list(set([item for sublist in drug_dates for item in sublist]))
            case.loc[-1] = [case_id, drug, flat_dates]  # adding a row
            case.index = case.index + 1  # shifting index
            case.sort_index(inplace=True) 
        case = case.drop_duplicates(subset=['std_drug'], keep='first')
    for combo in list(combinations(case['dates'].values.tolist(), limit)):
        common = combo[0]
        for i in range(1,limit):
            common = list(set(common) & set(combo[i]))
        if len(common) > 0:
            poly.append(1)
            poly.append(max(common)-min(common) + datetime.timedelta(days=1))
            break;
    if len(poly)==1:
        poly.append(0)
        poly.append(0)
    polypharm.append(poly)

In [ ]:
prescription_polypharmacy = pd.DataFrame(polypharm, columns =['hadm_id', 'polypharmacy', 'duration'])
prescription_polypharmacy.drop(columns = ['duration']).head()

,hadm_id,polypharmacy
0,107064,1
1,143045,1
2,150750,1
3,163353,0
4,185777,1


As can be seen below, among 49130 'hadm_id's, 46179 were found to have at least 5 drugs prescribed for the same day. We could set a higher threshold for the minimum number of drugs required to be classified as polypharmacy. The polypharmacy variable can be used to explore adverse effects of polypharmacy in a hospital setting in the MIMIC-III dataset.

In [ ]:
prescription_polypharmacy['hadm_id'].nunique()

49130

In [ ]:
prescription_polypharmacy['polypharmacy'].value_counts()

1    46179
0     2951
Name: polypharmacy, dtype: int64